<a href="https://colab.research.google.com/github/nano2547/PS-Research/blob/main/AL021KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import statistics as stat
from scipy.spatial.distance import cdist
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
df = pd.read_csv("/content/drive/MyDrive/Event 3.csv", skiprows=[0,1,2,4]) # put whatever event 3 is
df

,Time,C_LTC1BUS1_LTC1BUS2_1_LTC1BUS2_A,C_LTC1BUS1_LTC1BUS2_1_LTC1BUS2_B,C_LTC1BUS1_LTC1BUS2_1_LTC1BUS2_C,C_LTC1BUS1_LTC1BUS2_1_LTC1BUS2_N,V_BUS20KV1_AN,V_BUS20KV1_BN,V_BUS20KV1_CN,C_WE919_WE910_1_WE910_A,C_WE919_WE910_1_WE910_B,...,V_2DBU17_CN,C_3DBX3_2DBU17_1_2DBU17_A,C_3DBX3_2DBU17_1_2DBU17_B,C_3DBX3_2DBU17_1_2DBU17_C,V_DBU16_AN,V_DBU16_BN,V_DBU16_CN,C_DBU15_DBU16_1_DBU16_A,C_DBU15_DBU16_1_DBU16_B,C_DBU15_DBU16_1_DBU16_C
0,0,0.00408,0.000823,-0.00519,-0.000343,0.0451,0.0673,-0.245,-0.00055,0.000088,...,-0.0233,0.000081,-0.000039,-0.000175,-0.0811,-0.0916,-0.0558,0.000921,0.000245,0.000331
1,208,0.00408,0.000823,-0.00519,-0.000343,0.0451,0.0673,-0.245,-0.00055,0.000088,...,-0.0233,0.000081,-0.000039,-0.000175,-0.0811,-0.0916,-0.0558,0.000921,0.000245,0.000331
2,417,-2.95000,0.128000,2.90000,-0.086100,185.0000,9.0200,-192.000,-0.14800,0.076800,...,-14.1000,-0.010300,0.000550,0.010100,21.5000,0.5580,-21.4000,-0.087200,0.004190,0.084600
3,625,-59.60000,0.961000,60.30000,-1.700000,4820.0000,238.0000,-5050.000,-4.28000,2.170000,...,-1570.0000,-0.698000,0.047000,0.680000,1770.0000,22.6000,-1780.0000,-6.530000,0.264000,6.460000
4,833,-122.00000,-8.760000,132.00000,-1.750000,11400.0000,1070.0000,-12400.000,-13.10000,6.150000,...,-7040.0000,-2.320000,0.005850,2.380000,6820.0000,40.5000,-7330.0000,-30.200000,-0.264000,30.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96001,20000080,-107.00000,2.910000,104.00000,0.142000,10000.0000,-246.0000,-9790.000,-31.00000,23.600000,...,-7580.0000,-13.300000,6.150000,7.110000,9700.0000,-1940.0000,-7740.0000,-124.000000,36.600000,87.400000
96002,20000289,-102.00000,-6.680000,109.00000,0.152000,9570.0000,648.0000,-10200.000,-31.60000,21.800000,...,-8070.0000,-13.100000,5.200000,7.940000,9390.0000,-1130.0000,-8230.0000,-121.000000,26.800000,94.200000
96003,20000497,-96.80000,-16.200000,113.00000,0.159000,9050.0000,1540.0000,-10600.000,-32.00000,19.800000,...,-8520.0000,-13.000000,4.220000,8.730000,9020.0000,-319.0000,-8670.0000,-117.000000,16.800000,100.000000
96004,20000705,-90.60000,-25.700000,116.00000,0.169000,8470.0000,2420.0000,-10900.000,-32.20000,17.700000,...,-8900.0000,-12.700000,3.210000,9.460000,8590.0000,498.0000,-9050.0000,-113.000000,6.730000,106.000000


In [20]:
# remove first 0.25 seconds to allow system to stabilise before using data
data = df[df['Time']> 0.25e6].iloc[:, 1:]
data

,C_LTC1BUS1_LTC1BUS2_1_LTC1BUS2_A,C_LTC1BUS1_LTC1BUS2_1_LTC1BUS2_B,C_LTC1BUS1_LTC1BUS2_1_LTC1BUS2_C,C_LTC1BUS1_LTC1BUS2_1_LTC1BUS2_N,V_BUS20KV1_AN,V_BUS20KV1_BN,V_BUS20KV1_CN,C_WE919_WE910_1_WE910_A,C_WE919_WE910_1_WE910_B,C_WE919_WE910_1_WE910_C,...,V_2DBU17_CN,C_3DBX3_2DBU17_1_2DBU17_A,C_3DBX3_2DBU17_1_2DBU17_B,C_3DBX3_2DBU17_1_2DBU17_C,V_DBU16_AN,V_DBU16_BN,V_DBU16_CN,C_DBU15_DBU16_1_DBU16_A,C_DBU15_DBU16_1_DBU16_B,C_DBU15_DBU16_1_DBU16_C
1201,-104.0,-3.02,107.0,0.144,9770.0,307.0,-10100.0,-30.1,21.1,9.02,...,-8230.0,-13.4,5.26,8.12,9580.0,-1220.0,-8330.0,-86.4,20.00,66.5
1202,-99.1,-12.60,112.0,0.164,9260.0,1200.0,-10500.0,-30.6,19.2,11.30,...,-8690.0,-13.2,4.26,8.92,9210.0,-389.0,-8780.0,-83.9,12.90,71.0
1203,-93.1,-22.20,115.0,0.181,8710.0,2090.0,-10800.0,-30.8,17.2,13.50,...,-9080.0,-12.9,3.24,9.66,8780.0,442.0,-9180.0,-80.8,5.73,75.1
1204,-86.5,-31.60,118.0,0.196,8090.0,2970.0,-11100.0,-30.8,15.1,15.70,...,-9430.0,-12.5,2.19,10.40,8300.0,1270.0,-9520.0,-77.3,-1.46,78.7
1205,-79.4,-40.80,120.0,0.210,7430.0,3820.0,-11300.0,-30.7,13.0,17.70,...,-9710.0,-12.1,1.14,11.00,7770.0,2090.0,-9800.0,-73.2,-8.63,81.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96001,-107.0,2.91,104.0,0.142,10000.0,-246.0,-9790.0,-31.0,23.6,7.40,...,-7580.0,-13.3,6.15,7.11,9700.0,-1940.0,-7740.0,-124.0,36.60,87.4
96002,-102.0,-6.68,109.0,0.152,9570.0,648.0,-10200.0,-31.6,21.8,9.84,...,-8070.0,-13.1,5.20,7.94,9390.0,-1130.0,-8230.0,-121.0,26.80,94.2
96003,-96.8,-16.20,113.0,0.159,9050.0,1540.0,-10600.0,-32.0,19.8,12.20,...,-8520.0,-13.0,4.22,8.73,9020.0,-319.0,-8670.0,-117.0,16.80,100.0
96004,-90.6,-25.70,116.0,0.169,8470.0,2420.0,-10900.0,-32.2,17.7,14.50,...,-8900.0,-12.7,3.21,9.46,8590.0,498.0,-9050.0,-113.0,6.73,106.0


In [21]:
test_path = "Attack_Faults_001.csv"
df_test = pd.read_csv("/content/drive/MyDrive/Event 4.CSV", skiprows=[0,1,2,4]) # put whatever event 1 is

df_test

,Time,C_LTC1BUS1_LTC1BUS2_1_LTC1BUS2_A,C_LTC1BUS1_LTC1BUS2_1_LTC1BUS2_B,C_LTC1BUS1_LTC1BUS2_1_LTC1BUS2_C,C_LTC1BUS1_LTC1BUS2_1_LTC1BUS2_N,V_BUS20KV1_AN,V_BUS20KV1_BN,V_BUS20KV1_CN,C_WE919_WE910_1_WE910_A,C_WE919_WE910_1_WE910_B,...,V_2DBU17_CN,C_3DBX3_2DBU17_1_2DBU17_A,C_3DBX3_2DBU17_1_2DBU17_B,C_3DBX3_2DBU17_1_2DBU17_C,V_DBU16_AN,V_DBU16_BN,V_DBU16_CN,C_DBU15_DBU16_1_DBU16_A,C_DBU15_DBU16_1_DBU16_B,C_DBU15_DBU16_1_DBU16_C
0,0,-0.004329,-0.001515,-0.001623,-0.000095,0.04508,0.067282,-0.244665,-0.000550,0.000310,...,-0.023287,0.000081,-0.000039,-0.000175,0.006976,-0.170016,0.009497,-0.001304,0.000744,0.001239
1,208,-0.004329,-0.001515,-0.001623,-0.000095,0.04508,0.067282,-0.244665,-0.000550,0.000310,...,-0.023287,0.000081,-0.000039,-0.000175,0.006976,-0.170016,0.009497,-0.001304,0.000744,0.001239
2,417,-2.949805,0.129254,2.902578,-0.086547,185.03150,9.019005,-191.888600,-0.147661,0.077044,...,-14.103820,-0.010343,0.000550,0.010053,21.577300,0.534447,-21.554240,-0.085252,0.004687,0.085501
3,625,-59.594590,0.953098,60.338350,-1.701627,4822.05300,238.183100,-5049.003000,-4.277356,2.171564,...,-1571.959000,-0.697677,0.047035,0.680220,1767.359000,22.725030,-1783.271000,-6.532479,0.264938,6.454980
4,833,-121.711400,-8.763029,132.230300,-1.750747,11365.89000,1068.903000,-12438.870000,-13.147550,6.146612,...,-7043.420000,-2.322523,0.005846,2.382541,6816.230000,40.688840,-7326.779000,-30.226880,-0.263450,30.871960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96001,20000080,-107.363700,2.914632,104.303300,0.141372,10036.55000,-246.284100,-9789.851000,-31.042110,23.562030,...,-7589.041000,-13.282080,6.172494,7.107059,9714.433000,-1952.942000,-7746.662000,-123.451400,34.480090,88.890520
96002,20000289,-102.389100,-6.670727,108.918900,0.151196,9573.86300,647.097800,-10221.420000,-31.617610,21.703120,...,-8081.860000,-13.168070,5.220430,7.944878,9404.315000,-1143.866000,-8235.304000,-120.328600,24.622090,95.589390
96003,20000497,-96.799230,-16.229930,112.860300,0.159055,9051.57400,1536.183000,-10589.430000,-31.997740,19.709110,...,-8524.224000,-12.973920,4.237179,8.733336,9035.498000,-327.745300,-8672.682000,-116.471100,14.614240,101.690800
96004,20000705,-90.581010,-25.684520,116.101600,0.166915,8472.77400,2416.316000,-10891.430000,-32.181340,17.592980,...,-8913.785000,-12.700290,3.229803,9.466650,8610.810000,490.488500,-9056.354000,-111.891800,4.527538,107.160200


In [22]:
data_test = df_test[df_test['Time']> 0.25e6].iloc[:, 1:]

data_test

,C_LTC1BUS1_LTC1BUS2_1_LTC1BUS2_A,C_LTC1BUS1_LTC1BUS2_1_LTC1BUS2_B,C_LTC1BUS1_LTC1BUS2_1_LTC1BUS2_C,C_LTC1BUS1_LTC1BUS2_1_LTC1BUS2_N,V_BUS20KV1_AN,V_BUS20KV1_BN,V_BUS20KV1_CN,C_WE919_WE910_1_WE910_A,C_WE919_WE910_1_WE910_B,C_WE919_WE910_1_WE910_C,...,V_2DBU17_CN,C_3DBX3_2DBU17_1_2DBU17_A,C_3DBX3_2DBU17_1_2DBU17_B,C_3DBX3_2DBU17_1_2DBU17_C,V_DBU16_AN,V_DBU16_BN,V_DBU16_CN,C_DBU15_DBU16_1_DBU16_A,C_DBU15_DBU16_1_DBU16_B,C_DBU15_DBU16_1_DBU16_C
1201,-104.45750,-3.022276,107.3242,0.145302,9766.355,306.5743,-10073.130,-30.13120,21.10978,9.022247,...,-8232.835,-13.37720,5.258677,8.115199,9578.646,-1217.1300,-8334.986,-86.45124,19.949390,66.48052
1202,-99.07707,-12.646870,111.5508,0.162985,9264.375,1202.4630,-10467.260,-30.55253,19.23790,11.314330,...,-8685.367,-13.18175,4.261304,8.916998,9209.828,-388.6814,-8784.569,-83.89921,12.879240,71.00006
1203,-93.08139,-22.179920,115.0774,0.180668,8705.442,2090.8320,-10796.350,-30.78555,17.24713,13.538670,...,-9083.925,-12.90552,3.237452,9.664098,8784.433,442.2328,-9180.040,-80.83090,5.730214,75.07913
1204,-86.50970,-31.582200,117.9038,0.196387,8092.647,2965.5940,-11058.440,-30.82910,15.14937,15.679100,...,-9426.551,-12.54914,2.193594,10.351160,8304.582,1269.9770,-9518.550,-77.26310,-1.458240,78.69476
1205,-79.41438,-40.788330,119.9912,0.212105,7430.405,3821.7370,-11252.550,-30.68199,12.95866,17.723470,...,-9710.900,-12.11459,1.135613,10.974640,7773.811,2089.6200,-9798.472,-73.22099,-8.630921,81.82396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96001,-107.36370,2.914632,104.3033,0.141372,10036.550,-246.2841,-9789.851,-31.04211,23.56203,7.479362,...,-7589.041,-13.28208,6.172494,7.107059,9714.433,-1952.9420,-7746.662,-123.45140,34.480090,88.89052
96002,-102.38910,-6.670727,108.9189,0.151196,9573.863,647.0978,-10221.420,-31.61761,21.70312,9.914006,...,-8081.860,-13.16807,5.220430,7.944878,9404.315,-1143.8660,-8235.304,-120.32860,24.622090,95.58939
96003,-96.79923,-16.229930,112.8603,0.159055,9051.574,1536.1830,-10589.430,-31.99774,19.70911,12.289000,...,-8524.224,-12.97392,4.237179,8.733336,9035.498,-327.7453,-8672.682,-116.47110,14.614240,101.69080
96004,-90.58101,-25.684520,116.1016,0.166915,8472.774,2416.3160,-10891.430,-32.18134,17.59298,14.588160,...,-8913.785,-12.70029,3.229803,9.466650,8610.810,490.4885,-9056.354,-111.89180,4.527538,107.16020


In [23]:
attack_dict = {52: (1.10, 2.10, 'CT', 'C_CTWE8_A'), 53: (1.10, 2.10, 'CT', 'C_CTWE8_B'),
                   54: (1.10, 2.10, 'CT', 'C_CTWE8_C'), #55: (1.10, 2.10, 'CT', 'C_CTWE8_N'),
                   #                56:(2.30,3.00,'PT','V_PT_DIST1_AN'), 57:(2.30,3.00,'PT','V_PT_DIST1_BN'), 58:(2.30,3.00,'PT','V_PT_DIST1_CN'),
                   5: (5.00, 5.90, 'Vphase', 'V_PTWE4_AN'), 66: (6.60, 7.80, 'Cphase', 'C_CT_DIST2_B'),
                   68: (9.00, 10.10, 'GPS', 'V_PT_FDR3_AN'), 69: (9.00, 10.10, 'GPS', 'V_PT_FDR3_BN'),
                   70: (9.00, 10.10, 'GPS', 'V_PT_FDR3_CN'), 71: (9.00, 10.10, 'GPS', 'C_CT_FDR3_A'),
                   72: (9.00, 10.10, 'GPS', 'C_CT_FDR3_B'), 73: (9.00, 10.10, 'GPS', 'C_CT_FDR3_C'),
                   36: (10.50, 11.70, 'GPS2', 'C_CTWE2_A'), 37: (10.50, 11.70, 'GPS2', 'C_CTWE2_B'),
                   38: (10.50, 11.70, 'GPS2', 'C_CTWE2_C'), #39: (10.50, 11.70, 'GPS', 'C_CTWE2_N'),
                   77: (16.20, 17.00, 'CT2', 'C_CT_FDR1_A'), 78: (16.20, 17.00, 'CT2', 'C_CT_FDR1_B'),
                   79: (16.20, 17.00, 'CT2', 'C_CT_FDR1_C'), 86: (17.40, 18.30, 'PT', 'V_PT_FDR12_AN'),
                   87: (17.40, 18.30, 'PT', 'V_PT_FDR12_BN'), 88: (17.40, 18.30, 'PT', 'V_PT_FDR12_CN')}

In [24]:
attack_list = ['CT', 'Vphase', 'Cphase', 'GPS', 'GPS2', 'CT2', 'PT']

In [25]:

all_attack_times = set()
for k, values in attack_dict.items():
    all_attack_times.add((values[0], values[1]))



In [26]:
cols = data.columns

print(all_attack_times)

all_attack_indices = set()

for start, end in all_attack_times:
    attack_indices = df_test.index[(df_test['Time'] >= start * 1e6) & (df_test['Time'] <= end * 1e6)] - 1201
    all_attack_indices.add((attack_indices[0], attack_indices[-1]))

all_attack_indice = sorted(list(all_attack_indices))

all_attack_indices

{(9.0, 10.1), (17.4, 18.3), (6.6, 7.8), (5.0, 5.9), (10.5, 11.7), (16.2, 17.0), (1.1, 2.1)}


{(4080, 8879),
 (22800, 27119),
 (30480, 36239),
 (42000, 47279),
 (49200, 54959),
 (76560, 80399),
 (82320, 86639)}

In [27]:
adj_list = {1: [29, 65],
                2: [30, 66],
                3: [31, 67],
                5: [12, 26, 19, 62],
                6: [13, 27, 20, 63],
                7: [14, 28, 21, 64],
                8: [15, 22],
                9: [16, 23],
                10: [17, 24],
                12: [5, 26, 33, 19],
                13: [6, 27, 34, 20],
                14: [7, 28, 35, 21],
                15: [8, 29, 52],
                16: [9, 53, 30],
                17: [10, 54, 31],
                19: [26, 5, 33, 12],
                20: [27, 34, 6, 13],
                21: [28, 35, 7, 14],
                22: [8, 36, 40, 29],
                23: [9, 37, 41, 30],
                24: [10, 38, 42, 31],
                26: [12, 33, 5, 19],
                27: [13, 20, 34, 6],
                28: [14, 21, 35, 7],
                29: [1, 65, 15, 48, 22],
                30: [2, 66, 49, 16, 23],
                31: [3, 67, 50, 17, 24],
                33: [56, 19, 26, 12],
                34: [57, 13, 27, 20],
                35: [58, 14, 28, 21],
                36: [22],
                37: [23],
                38: [24],
                40: [36, 48, 22],
                41: [37, 49, 23],
                42: [38, 50, 24],
                44: [52, 59],
                45: [53, 60],
                46: [54, 61],
                48: [40, 52, 29],
                49: [53, 30, 41],
                50: [31, 42, 54],
                52: [44, 59, 48, 15],
                53: [45, 60, 49, 16],
                54: [17, 50, 46, 61],
                56: [33, 74, 68],
                57: [34, 75, 69],
                58: [35, 76, 70],
                59: [71, 77, 44, 52],
                60: [45, 78, 72, 53],
                61: [46, 79, 73, 54],
                62: [5, 86, 80],
                63: [6, 87, 81],
                64: [7, 88, 82],
                65: [29, 1, 89, 83],
                66: [2, 30, 90, 84],
                67: [85, 91, 3, 31],
                68: [56, 74],
                69: [57, 75],
                70: [58, 76],
                71: [59, 77],
                72: [60, 78],
                73: [79, 61],
                74: [56, 68],
                75: [57, 69],
                76: [58, 70],
                77: [59, 71],
                78: [60, 72],
                79: [61, 73],
                80: [62, 86],
                81: [63, 87],
                82: [64, 88],
                83: [89, 65],
                84: [90, 66],
                85: [91, 67],
                86: [62, 80],
                87: [63, 81],
                88: [64, 82],
                89: [65, 83],
                90: [66, 84],
                91: [85, 67]}

In [28]:
neutral_sensors = []
emptyr = [4, 11, 18, 25, 32, 39, 43, 47, 51, 55]  # nodes without connections to anything
unmonitored_sensors = set(neutral_sensors + emptyr)

new_adj_list = {}
for sensors, neigh in adj_list.items():
  if sensors in unmonitored_sensors:
    continue
  else:
    new_nei = []
    for i in neigh:
      if i in unmonitored_sensors:
        continue
      else:
        new_nei += [i]
    new_adj_list[sensors] = new_nei

adj_list = new_adj_list

In [29]:
attacked_sensors = []
for index, i in enumerate(attack_list):
  attak_list = []
  for j, value in attack_dict.items():
    if value[2] == i:
      attak_list += [j]+adj_list[j]
      print(attak_list)
  attacked_sensors += [attak_list]
attacked_sensors

[52, 44, 59, 48, 15]
[52, 44, 59, 48, 15, 53, 45, 60, 49, 16]
[52, 44, 59, 48, 15, 53, 45, 60, 49, 16, 54, 17, 50, 46, 61]
[5, 12, 26, 19, 62]
[66, 2, 30, 90, 84]
[68, 56, 74]
[68, 56, 74, 69, 57, 75]
[68, 56, 74, 69, 57, 75, 70, 58, 76]
[68, 56, 74, 69, 57, 75, 70, 58, 76, 71, 59, 77]
[68, 56, 74, 69, 57, 75, 70, 58, 76, 71, 59, 77, 72, 60, 78]
[68, 56, 74, 69, 57, 75, 70, 58, 76, 71, 59, 77, 72, 60, 78, 73, 79, 61]
[36, 22]
[36, 22, 37, 23]
[36, 22, 37, 23, 38, 24]
[77, 59, 71]
[77, 59, 71, 78, 60, 72]
[77, 59, 71, 78, 60, 72, 79, 61, 73]
[86, 62, 80]
[86, 62, 80, 87, 63, 81]
[86, 62, 80, 87, 63, 81, 88, 64, 82]


[[52, 44, 59, 48, 15, 53, 45, 60, 49, 16, 54, 17, 50, 46, 61],
 [5, 12, 26, 19, 62],
 [66, 2, 30, 90, 84],
 [68, 56, 74, 69, 57, 75, 70, 58, 76, 71, 59, 77, 72, 60, 78, 73, 79, 61],
 [36, 22, 37, 23, 38, 24],
 [77, 59, 71, 78, 60, 72, 79, 61, 73],
 [86, 62, 80, 87, 63, 81, 88, 64, 82]]

# Method

Get the anomalous data, get K-Nearest distance from anomolous data to training data. Divide that by median and get the G-score. Collect the G-score for all sensors. Use percentiles to find out which sensor is attacked.

In [30]:
def medians (dist):
  #finds the median of each dist
  median_list = []
  for i in dist:
    medi = np.median(i)
    median_list += [medi]
  return median_list

**Test for all**

**Thresholding and checking every sensor**

In [31]:
col = data.columns

In [32]:
#count of sensors in total attack neighborhood
c = 0
for i in attacked_sensors:
  for sensors in i:
    c += 1

In [33]:
def g_scoring (dist_sensor, median_list):
  g_scores = []
  for k in [1,5]:
    k = int(k)
    dist_100 = np.partition(dist_sensor[:], kth=k-1, axis=1)[:,k-1]
    g_score = np.divide(dist_100, median_list)
    g_scores += [g_score]
  return g_scores

In [34]:
def multi_gscoring (g_scores, g_score1, g_score5):
  for ind, i in enumerate(g_scores):
    g_mean = stat.mean(i)
    if ind == 0:
      g_score1 += [g_mean]
    if ind == 1:
      g_score5 += [g_mean]
    #if ind == 2:
      #g_score10 += [g_mean]
    #if ind == 3:
      #g_score50 += [g_mean]
    #if ind == 4:
      #g_score100 += [g_mean]

In [35]:
def threshold_gscore (quantile, g_score1, g_score5):
  g_score1.sort()
  g_score5.sort()
  threshold1 = g_score1[-quantile]
  threshold5 = g_score5[-quantile]
  #threshold10 = np.quantile(g_score10, quantile)
  #threshold50 = np.quantile(g_score50, quantile)
  #threshold100 = np.quantile(g_score100, quantile)
  return threshold1, threshold5

Calculate LD (Localization Delay) FPR-L (False Positive Rate Localization) and FLR (False Localization Rate) and Time to run a window.

In [36]:
import time
start_time = time.time()
co = 0
flr = []
fpr_d = []
fpr_l = []
for start, end in sorted(all_attack_indice):
  g_score1 = []
  g_score5 = []
  g_score_dict = {}

  #calculating g_scores
  for index, columns in enumerate(col):
    sensor = index+1
    if sensor in unmonitored_sensors:
        continue #taking out empty rows
    else:
      nbrs = np.array(adj_list[sensor]+[sensor]) -1
      data_sensor=data.iloc[start:end, nbrs]
      data_test_sensor=data_test.iloc[start:end, nbrs]
      dist_sensor= cdist(data_sensor, data_test_sensor) #cdisting the whole thing
      median_list = medians(dist_sensor)
      g_scores = g_scoring(dist_sensor, median_list)
      multi_gscoring (g_scores, g_score1, g_score5)
      g_score_dict[sensor] = g_scores

  #thresholding
  num_sen = len(attacked_sensors[co])
  threshold1, threshold5 = threshold_gscore (num_sen, g_score1, g_score5)

  #finding the anomalies that crosses the thresholds
  anomaly1 = [i for i in g_score1 if i>= threshold1]
  anomaly5 = [i for i in g_score5 if i>= threshold5]
  #converting them to names
  anomaly1_names = []
  anomaly5_names = []

  for key, value in g_score_dict.items():
    for i in value:
      j=stat.mean(i)
      if j in anomaly1:
        anomaly1_names += [key]
      if j in anomaly5:
        anomaly5_names += [key]

  false_localised_sensors = []
  for anom in anomaly5_names:
    if anom in attacked_sensors[co]:
      continue
    else:
      false_localised_sensors += [anom]

  #FLR
  print(f"For {attack_list[co]} attack, the sensors that are detected to be anomalous is:")
  print(f"K=1 {anomaly1_names}")
  print(f"K=5 {anomaly5_names}")

  print(f"1 Hop: {attacked_sensors[co]}")
  print(f"False Localisations: {false_localised_sensors}")

  flr.append(len(false_localised_sensors)/num_sen)

  #FPR_L
  timesteps = end-start
  false_positives = len(false_localised_sensors) * timesteps
  total_negatives = timesteps * (len(cols)-len(attacked_sensors[co]))
  FPR_L = false_positives / total_negatives
  fpr_l.append(FPR_L)

  #FPR_D
  total_t = 94804
  negatives = total_t *(len(cols)-len(attacked_sensors[co]))
  FPR_D = false_positives/negatives
  fpr_d.append(FPR_D)

  #LD

  co += 1

total_time = time.time() - start_time

For CT attack, the sensors that are detected to be anomalous is:
K=1 [15, 16, 17, 44, 45, 46, 48, 49, 50, 52, 53, 54, 59, 60, 61]
K=5 [15, 16, 17, 44, 45, 46, 48, 49, 50, 52, 53, 54, 59, 60, 61]
1 Hop: [52, 44, 59, 48, 15, 53, 45, 60, 49, 16, 54, 17, 50, 46, 61]
False Localisations: []
For Vphase attack, the sensors that are detected to be anomalous is:
K=1 [5, 12, 19, 26, 62]
K=5 [5, 12, 19, 26, 62]
1 Hop: [5, 12, 26, 19, 62]
False Localisations: []
For Cphase attack, the sensors that are detected to be anomalous is:
K=1 [2, 30, 66, 84, 90]
K=5 [2, 30, 66, 84, 90]
1 Hop: [66, 2, 30, 90, 84]
False Localisations: []
For GPS attack, the sensors that are detected to be anomalous is:
K=1 [56, 57, 58, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 83, 84, 89, 90]
K=5 [56, 57, 58, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 83, 84, 89, 90]
1 Hop: [68, 56, 74, 69, 57, 75, 70, 58, 76, 71, 59, 77, 72, 60, 78, 73, 79, 61]
False Localisations: [83, 84, 89, 90]
For GPS2 attack, the sensors th

In [37]:
(np.mean(fpr_d)*100, np.std(fpr_l)*100)

(0.20913965977994503, 4.574412795878916)

In [38]:
(np.mean(fpr_l)*100,np.std(fpr_l)*100)

(4.422869047385075, 4.574412795878916)

In [39]:
flr

[0.0, 0.0, 0.0, 0.2222222222222222, 0.5, 1.0, 1.0]

In [40]:
np.mean(flr), np.std(flr)


(0.3888888888888889, 0.4210091177738046)

In [41]:
time_a = 0
for star, en in all_attack_indices:
  time_a += (en-star)
time_a

34073

In [42]:
total_time*1000/time_a

14.032413152505999